In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,abatement,ablaze,able,abridge,absence,absorbed,abumayyaleh,abuse,abused,abusing,...,year,yearning,years,york,yorkers,yorks,young,younger,youre,zimmerman
1,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,0,0,1,0,0,0
2,7,0,0,0,0,0,2,0,0,0,...,0,0,0,4,2,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
5,0,1,0,0,1,0,0,2,1,0,...,3,0,1,1,0,0,1,0,0,0
6,0,0,0,0,0,0,0,2,0,0,...,1,0,2,1,0,0,3,1,1,1
7,0,0,0,1,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,2,0
8,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,1,2,3,4,5,6,7,8
abatement,0,7,0,0,0,0,0,0
ablaze,0,0,0,0,1,0,0,0
able,0,0,0,1,0,0,0,1
abridge,0,0,0,0,0,0,1,0
absence,0,0,0,0,1,0,1,0


In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.010*"black" + 0.007*"racial" + 0.007*"people" + 0.005*"justice" + 0.005*"like" + 0.005*"white" + 0.004*"violence" + 0.004*"economic" + 0.004*"health" + 0.003*"nation"'),
 (1,
  '0.009*"officers" + 0.006*"black" + 0.005*"protesters" + 0.005*"country" + 0.005*"rage" + 0.004*"justice" + 0.004*"americans" + 0.004*"violence" + 0.004*"people" + 0.004*"like"'),
 (2,
  '0.006*"store" + 0.005*"officers" + 0.005*"stores" + 0.005*"court" + 0.005*"courts" + 0.004*"justice" + 0.004*"americans" + 0.004*"immunity" + 0.004*"new" + 0.004*"nuisance"')]

In [8]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.008*"confederate" + 0.006*"statues" + 0.005*"speaker" + 0.004*"states" + 0.004*"state" + 0.004*"did" + 0.004*"slavery" + 0.004*"house" + 0.004*"confederacy" + 0.003*"mr"'),
 (1,
  '0.012*"black" + 0.008*"racial" + 0.008*"people" + 0.007*"justice" + 0.005*"like" + 0.005*"white" + 0.005*"violence" + 0.005*"health" + 0.005*"economic" + 0.004*"nation"'),
 (2,
  '0.010*"officers" + 0.005*"justice" + 0.005*"americans" + 0.005*"country" + 0.005*"store" + 0.004*"law" + 0.004*"stores" + 0.004*"court" + 0.004*"courts" + 0.004*"new"'),
 (3,
  '0.008*"black" + 0.006*"rage" + 0.005*"officers" + 0.005*"protesters" + 0.004*"violence" + 0.004*"justice" + 0.004*"people" + 0.004*"hope" + 0.004*"psalm" + 0.003*"americans"')]

In [9]:
# Topic Modeling - Attempt #2 (Nouns Only)
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [10]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
1,confederate statues are being pulled down acro...
2,ever since george floyd was killed by minneapo...
3,there are videos of eric garner and george flo...
4,stop killing us three words scrawled on a sign...
5,a minneapolis police officer who was filmed kn...
6,our democracy hangs in the balance this is not...
7,when george floyd died under the knee of a min...
8,imagine that no one had shot video of george f...


In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Topic Modeling - Attempt #3 (Nouns and Adjectives)
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))